# Demonstração de Algoritmos de Machine Learning

Este notebook demonstra o uso dos algoritmos de machine learning implementados neste repositório.
Vamos usar os conjuntos de dados sintéticos gerados nos scripts da pasta `data` para testar os diversos algoritmos.

In [ ]:
# Importações necessárias
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
import seaborn as sns

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Carregando os Dados

Vamos importar os geradores de dados que criamos para poder testar os algoritmos com diferentes tipos de conjuntos de dados.

In [ ]:
# Importar geradores de dados
from data.classificacao import gerar_classificacao_linear, gerar_classificacao_nao_linear, gerar_xor, gerar_multiclasse, visualizar
from data.regressao import gerar_regressao_linear, gerar_regressao_polinomial, gerar_regressao_senoidal, gerar_regressao_multipla, visualizar_regressao
from data.clustering import gerar_clusters, gerar_clusters_nao_lineares, gerar_dados_alta_dimensao, visualizar_clusters
from data.reinforcement import AmbienteGrade, criar_ambiente_padrao

## 1. Algoritmos de Aprendizado Supervisionado - Classificação

### 1.1 K-Nearest Neighbors (KNN)

In [ ]:
# Importar algoritmo KNN
from supervised.knn.knn import KNN

# Gerar dados para classificação linear
X, y = gerar_classificacao_linear(n_amostras=200, n_atributos=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar(X_train, y_train, "Dados de Treinamento")

# Treinar e testar KNN
knn = KNN(k=3)
knn.treinar(X_train, y_train)
y_pred = knn.prever(X_test)

# Avaliar desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do KNN: {accuracy:.4f}")

# Visualizar predições
plt.subplot(1, 2, 2)
visualizar(X_test, y_pred, "Predições do KNN")
plt.tight_layout()

# Visualizar fronteira de decisão
def visualizar_fronteira_decisao(X, y, modelo, titulo, ax=None):
    # Definir limites para os eixos
    h = 0.02  # Tamanho do passo na malha
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # Prever rótulos para cada ponto na malha
    Z = modelo.prever(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))
        
    # Plotar a fronteira de decisão
    ax.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
    
    # Plotar os pontos
    cores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    for i in np.unique(y):
        ax.scatter(X[y == i, 0], X[y == i, 1], 
                  color=cores[int(i) % len(cores)], 
                  label=f'Classe {i}', 
                  edgecolors='k')
    
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_title(titulo)
    ax.set_xlabel('Atributo 1')
    ax.set_ylabel('Atributo 2')
    ax.legend()
    
    return ax

plt.figure(figsize=(12, 10))
visualizar_fronteira_decisao(X, y, knn, f"Fronteira de Decisão do KNN (k={knn.k})")
plt.tight_layout()

### 1.2 Naive Bayes

In [ ]:
# Importar algoritmo Naive Bayes
from supervised.naive_bayes.naive_bayes import NaiveBayes

# Gerar dados para classificação multiclasse
X, y = gerar_classificacao_linear(n_amostras=200, n_atributos=2, n_classes=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar(X_train, y_train, "Dados de Treinamento")

# Treinar e testar Naive Bayes
nb = NaiveBayes()
nb.treinar(X_train, y_train)
y_pred = nb.prever(X_test)

# Avaliar desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Naive Bayes: {accuracy:.4f}")

# Visualizar predições
plt.subplot(1, 2, 2)
visualizar(X_test, y_pred, "Predições do Naive Bayes")
plt.tight_layout()

# Visualizar fronteira de decisão
plt.figure(figsize=(12, 10))
visualizar_fronteira_decisao(X, y, nb, "Fronteira de Decisão do Naive Bayes")
plt.tight_layout()

### 1.3 Árvore de Decisão

In [ ]:
# Importar algoritmo de Árvore de Decisão
from supervised.decision_tree.decision_tree import DecisionTree

# Gerar dados XOR (não linearmente separáveis)
X, y = gerar_xor(n_amostras=200, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar(X_train, y_train, "Dados de Treinamento (XOR)")

# Treinar e testar Árvore de Decisão
dt = DecisionTree(max_depth=5)
dt.treinar(X_train, y_train)
y_pred = dt.prever(X_test)

# Avaliar desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia da Árvore de Decisão: {accuracy:.4f}")

# Visualizar predições
plt.subplot(1, 2, 2)
visualizar(X_test, y_pred, "Predições da Árvore de Decisão")
plt.tight_layout()

# Visualizar fronteira de decisão
plt.figure(figsize=(12, 10))
visualizar_fronteira_decisao(X, y, dt, f"Fronteira de Decisão da Árvore (profundidade={dt.max_depth})")
plt.tight_layout()

### 1.4 Support Vector Machine (SVM)

In [ ]:
# Importar algoritmo SVM
from supervised.svm.svm import SVM

# Gerar dados para classificação não linear
X, y = gerar_classificacao_nao_linear('moons', n_amostras=200, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar(X_train, y_train, "Dados de Treinamento (Moons)")

# Treinar e testar SVM
svm = SVM(learning_rate=0.01, n_iters=1000, kernel='rbf')
svm.treinar(X_train, y_train)
y_pred = svm.prever(X_test)

# Avaliar desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do SVM: {accuracy:.4f}")

# Visualizar predições
plt.subplot(1, 2, 2)
visualizar(X_test, y_pred, "Predições do SVM")
plt.tight_layout()

# Visualizar fronteira de decisão
plt.figure(figsize=(12, 10))
visualizar_fronteira_decisao(X, y, svm, f"Fronteira de Decisão do SVM (kernel={svm.kernel})")
plt.tight_layout()

## 2. Algoritmos de Aprendizado Supervisionado - Regressão

### 2.1 Regressão Linear

In [ ]:
# Importar algoritmo de Regressão Linear
from supervised.linear_regression.linear_regression import LinearRegression

# Gerar dados para regressão linear
X, y = gerar_regressao_linear(n_amostras=100, n_atributos=1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 8))
plt.scatter(X[:, 0], y, color='blue', alpha=0.7, label='Dados')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Dados de Regressão Linear')
plt.grid(True, linestyle='--', alpha=0.7)

# Treinar e testar Regressão Linear
lr = LinearRegression(learning_rate=0.01, n_iters=1000)
lr.treinar(X_train, y_train)
y_pred = lr.prever(X_test)

# Avaliar desempenho
mse = mean_squared_error(y_test, y_pred)
print(f"MSE da Regressão Linear: {mse:.4f}")
print(f"Coeficientes: {lr.coef_}")
print(f"Intercepto: {lr.intercept_}")

# Visualizar a linha de regressão
x_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 100).reshape(-1, 1)
y_pred_line = lr.prever(x_range)
plt.plot(x_range, y_pred_line, color='red', linewidth=3, label='Linha de Regressão')
plt.scatter(X_test[:, 0], y_test, color='green', marker='x', s=100, label='Teste')
plt.scatter(X_test[:, 0], y_pred, color='orange', marker='o', s=100, label='Predições')
plt.legend()
plt.tight_layout()

## 3. Algoritmos de Aprendizado Não Supervisionado

### 3.1 K-Means

In [ ]:
# Importar algoritmo K-Means
from unsupervised.kmeans.kmeans import KMeans

# Gerar dados para clustering
X, y_true = gerar_clusters(n_amostras=300, n_clusters=4, random_state=42)

# Visualizar dados originais
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar_clusters(X, y_true, "Clusters Originais")

# Treinar K-Means e obter clusters
kmeans = KMeans(K=4, max_iters=100, plot_steps=False)
y_pred = kmeans.prever(X)

# Visualizar clusters encontrados
plt.subplot(1, 2, 2)
visualizar_clusters(X, y_pred, "Clusters Encontrados pelo K-Means")
plt.tight_layout()

# Visualizar centroides
plt.figure(figsize=(10, 8))
ax = visualizar_clusters(X, y_pred, "Clusters e Centroides do K-Means")
centroides = kmeans.centroides
ax.scatter(centroides[:, 0], centroides[:, 1], 
           marker='*', s=300, c='black', label='Centroides')
plt.legend()
plt.tight_layout()

### 3.2 PCA (Principal Component Analysis)

In [ ]:
# Importar algoritmo PCA
from unsupervised.pca.pca import PCA

# Gerar dados de alta dimensão
X, y = gerar_dados_alta_dimensao(n_amostras=200, n_atributos=10, n_informative=2, n_clusters=3, random_state=42)

# Visualizar os dados originais (apenas as duas primeiras dimensões)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar_clusters(X[:, :2], y, "Dados Originais (2 primeiras dimensões)")

# Aplicar PCA
pca = PCA(n_componentes=2)
X_reduzido = pca.transformar(X)

# Visualizar dados após a redução de dimensionalidade
plt.subplot(1, 2, 2)
visualizar_clusters(X_reduzido, y, "Dados após PCA (2 componentes)")
plt.tight_layout()

# Visualizar variância explicada
plt.figure(figsize=(10, 6))
variancia_explicada = pca.variancia_explicada
plt.bar(range(len(variancia_explicada)), variancia_explicada, alpha=0.7)
plt.plot(range(len(variancia_explicada)), np.cumsum(variancia_explicada), marker='o', linestyle='-', color='red')
plt.xlabel('Componente Principal')
plt.ylabel('Proporção de Variância Explicada')
plt.title('Variância Explicada por Componente')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

## 4. Aprendizado por Reforço

### 4.1 Q-Learning

In [ ]:
# Importar algoritmo Q-Learning
from reinforcement.qlearning.qlearning import QLearning

# Criar ambiente de grade
ambiente = criar_ambiente_padrao()

# Visualizar ambiente inicial
plt.figure(figsize=(10, 10))
ambiente.renderizar()
plt.tight_layout()

# Treinar agente Q-Learning
qlearner = QLearning(
    num_estados=ambiente.obter_espaco_estados(),
    num_acoes=ambiente.obter_espaco_acoes(),
    taxa_aprendizado=0.1,
    fator_desconto=0.99,
    epsilon=0.1
)

# Função auxiliar para converter posição (x, y) para índice de estado
def posicao_para_estado(posicao, largura):
    x, y = posicao
    return y * largura + x

# Função auxiliar para converter índice de estado para posição (x, y)
def estado_para_posicao(estado, largura):
    x = estado % largura
    y = estado // largura
    return (x, y)

# Treinar o agente
num_episodios = 500
max_passos = 100
recompensas_por_episodio = []

for episodio in range(num_episodios):
    # Reiniciar ambiente
    posicao_atual = ambiente.reset()
    estado_atual = posicao_para_estado(posicao_atual, ambiente.largura)
    terminado = False
    recompensa_total = 0
    passos = 0
    
    while not terminado and passos < max_passos:
        # Escolher ação usando política epsilon-greedy
        acao = qlearner.selecionar_acao(estado_atual)
        
        # Executar ação no ambiente
        nova_posicao, recompensa, terminado = ambiente.passo(acao)
        novo_estado = posicao_para_estado(nova_posicao, ambiente.largura)
        
        # Atualizar Q-table
        qlearner.atualizar(estado_atual, acao, recompensa, novo_estado, terminado)
        
        # Atualizar estado atual
        estado_atual = novo_estado
        recompensa_total += recompensa
        passos += 1
    
    recompensas_por_episodio.append(recompensa_total)
    
    # Imprimir progresso a cada 100 episódios
    if (episodio + 1) % 100 == 0:
        print(f"Episódio {episodio + 1}/{num_episodios}, Recompensa: {recompensa_total:.2f}, Passos: {passos}")

# Visualizar curva de aprendizado
plt.figure(figsize=(12, 6))
plt.plot(recompensas_por_episodio, label='Recompensa por Episódio')
plt.xlabel('Episódio')
plt.ylabel('Recompensa Total')
plt.title('Curva de Aprendizado do Q-Learning')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()

# Extrair a política aprendida
politica = np.zeros((ambiente.altura, ambiente.largura), dtype=int)
for y in range(ambiente.altura):
    for x in range(ambiente.largura):
        estado = posicao_para_estado((x, y), ambiente.largura)
        politica[y, x] = np.argmax(qlearner.Q[estado, :])

# Visualizar a política aprendida
plt.figure(figsize=(10, 10))
ambiente.renderizar(politica=politica)
plt.title('Política Aprendida pelo Q-Learning')
plt.tight_layout()

## 5. Deep Learning

### 5.1 Rede Neural

In [ ]:
# Importar algoritmo de Rede Neural
from deep_learning.neural_network.neural_network import NeuralNetwork, Layer

# Função para normalizar os dados
def normalizar(X):
    return (X - X.mean(axis=0)) / X.std(axis=0)

# Gerar dados para classificação não linear
X, y = gerar_classificacao_nao_linear('moons', n_amostras=300, random_state=42)
X = normalizar(X)  # Normalizar para melhor convergência da rede neural
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizar dados
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
visualizar(X_train, y_train, "Dados de Treinamento (Moons)")

# Converter rótulos para one-hot encoding para classificação com rede neural
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

# Converter de one-hot encoding para rótulos
def from_categorical(y_cat):
    return np.argmax(y_cat, axis=1)

# Converter rótulos para formato adequado
y_train_cat = to_categorical(y_train, num_classes=2)
num_classes = y_train_cat.shape[1]

# Criar e treinar rede neural
nn = NeuralNetwork()
nn.add(Layer(2, 16, activation='relu'))  # Camada de entrada: 2 neurônios (atributos), 16 neurônios na camada oculta
nn.add(Layer(16, 8, activation='relu'))  # Segunda camada oculta
nn.add(Layer(8, num_classes, activation='softmax'))  # Camada de saída: num_classes neurônios

# Treinar a rede
historia = nn.treinar(X_train, y_train_cat, learning_rate=0.1, n_epochs=1000, batch_size=32)

# Fazer predições
y_pred_cat = nn.prever(X_test)
y_pred = from_categorical(y_pred_cat)

# Avaliar desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia da Rede Neural: {accuracy:.4f}")

# Visualizar predições
plt.subplot(1, 2, 2)
visualizar(X_test, y_pred, "Predições da Rede Neural")
plt.tight_layout()

# Visualizar fronteira de decisão
class NeuralNetworkWrapper:
    def __init__(self, model):
        self.model = model
    
    def prever(self, X):
        y_pred = self.model.prever(X)
        return from_categorical(y_pred)

nn_wrapper = NeuralNetworkWrapper(nn)
plt.figure(figsize=(12, 10))
visualizar_fronteira_decisao(X, y, nn_wrapper, "Fronteira de Decisão da Rede Neural")
plt.tight_layout()

# Visualizar histórico de perda (loss)
plt.figure(figsize=(12, 6))
plt.plot(historia['loss'], label='Perda (Loss)')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.title('Histórico de Treinamento da Rede Neural')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()

## Conclusão

Neste notebook, demonstramos o uso de diversos algoritmos de machine learning implementados do zero:

1. **Algoritmos de Classificação Supervisionada**:
   - K-Nearest Neighbors (KNN)
   - Naive Bayes
   - Árvore de Decisão
   - Support Vector Machine (SVM)

2. **Algoritmos de Regressão Supervisionada**:
   - Regressão Linear

3. **Algoritmos de Aprendizado Não Supervisionado**:
   - K-Means
   - PCA (Principal Component Analysis)

4. **Algoritmos de Aprendizado por Reforço**:
   - Q-Learning

5. **Deep Learning**:
   - Rede Neural Feed-Forward

Cada algoritmo foi testado com conjuntos de dados adequados para demonstrar suas capacidades e limitações.
Esta implementação serve principalmente para fins educacionais, permitindo compreender os fundamentos desses algoritmos.
Para aplicações em produção, recomenda-se o uso de bibliotecas como scikit-learn, TensorFlow ou PyTorch.